In [1]:
import tensorflow as tf
import time
from tensorflow.python.client import timeline
#from tensorflow.python.client import device_lib

In [2]:
# "k" is the filename; "v" the value; "t" the record
def view_data(k,v,t):
    print('Key:        {}'.format(k))
    count = 0
    for x in t:
        count += 1
    print('Count: {}'.format(count))

In [3]:
'''with tf.device('/cpu:0'):
    files  = tf.train.match_filenames_once("/home/tom/data/rectangles/rectangles-images_val.txt")
    file_q = tf.train.string_input_producer(files, num_epochs=1)

    reader     = tf.TextLineReader(skip_header_lines=1)
    key, value = reader.read(file_q)'''

# For the "rectangles" data set, 28x28 columns of pixel values + the last
# column is the label: 0 or 1
files  = tf.train.match_filenames_once("/home/tom/data/rectangles/rectangles-images_val.txt")
file_q = tf.train.string_input_producer(files, num_epochs=1)

reader     = tf.TextLineReader(skip_header_lines=1)
key, value = reader.read(file_q)

defaults = [[0.] for x in range(785)]

T = tf.decode_csv(value, record_defaults=defaults, name='decode_op')

'''with tf.device('/cpu:0'):
    tom = tf.reduce_sum(T, name='Tom')'''

"with tf.device('/cpu:0'):\n    tom = tf.reduce_sum(T, name='Tom')"

In [4]:
# turn off graph optimizations
'''no_opt = tf.OptimizerOptions(opt_level=tf.OptimizerOptions.L0,
                             do_common_subexpression_elimination=False,
                             do_function_inlining=False,
                             do_constant_folding=False)
config = tf.ConfigProto(graph_options=tf.GraphOptions(optimizer_options=no_opt),
                        log_device_placement=True,
                        allow_soft_placement=True,
                        inter_op_parallelism_threads=3,
                        intra_op_parallelism_threads=3)'''

config = tf.ConfigProto(log_device_placement=True,
                        allow_soft_placement=True)

x = 0
keys = []
sv_file = ''

with tf.Session(config=config) as sess:
    metadata = tf.RunMetadata()
    options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE,
                            output_partition_graphs=True)
    coord = tf.train.Coordinator()
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    threads = tf.train.start_queue_runners(coord=coord)
    
    start_time = time.time()
    total_time = 0
    read_time = time.time()
    
    try:
        while not coord.should_stop():
            k,v,t = sess.run([key,value,T], options=options, run_metadata=metadata)
            
            total_time += time.time() - read_time
            read_time = time.time()
            
            break
            
            offset = str(k).find(':')
            filenm = k[:offset-2]
            if filenm != sv_file:
                print('Key {} has {} records'.format(sv_file, x))
                sv_file = k[:offset-2]
                x=0
            #view_data(k,v,t)
            x += 1
            j = sess.run(T, options=options, run_metadata=metadata)
    except tf.errors.OutOfRangeError:
        print('Key {} has {} records'.format(sv_file, x))
    finally:
        coord.request_stop()
    coord.join(threads)
    print('Duration: {:.1f} minutes'.format((time.time() - start_time)/60))
    print('KVT elapsed time: {}'.format(total_time))

Duration: 0.0 minutes
KVT elapsed time: 0.10682177543640137


In [5]:
#sess.run(res.op, options=options, run_metadata=metadata)

'''trace = timeline.Timeline(step_stats=metadata.step_stats)
with open('/tmp/timeline.json', 'w') as out:
    out.write(trace.generate_chrome_trace_format())'''

#print(str(metadata))

"trace = timeline.Timeline(step_stats=metadata.step_stats)\nwith open('/tmp/timeline.json', 'w') as out:\n    out.write(trace.generate_chrome_trace_format())"

In [6]:
#your_writer.add_run_metadata(run_metadata, 'step%d' % i)

In [7]:
len(t)

785

In [8]:
for x in t: print(x)

0.219608
0.54902
0.698039
0.45098
0.117647
0.156863
0.223529
0.129412
0.0862745
0.156863
0.415686
0.768627
0.521569
0.172549
0.196078
0.392157
0.764706
0.556863
0.0705882
0.0862745
0.176471
0.137255
0.4
0.494118
0.231373
0.513726
0.690196
0.65098
0.121569
0.184314
0.388235
0.690196
0.678431
0.309804
0.0823529
0.129412
0.188235
0.113725
0.227451
0.333333
0.713726
0.721569
0.231373
0.145098
0.337255
0.705882
0.541176
0.192157
0.141176
0.0901961
0.137255
0.454902
0.447059
0.258824
0.411765
0.729412
0.203922
0.121569
0.0901961
0.239216
0.54902
0.67451
0.470588
0.219608
0.137255
0.141176
0.0980392
0.133333
0.262745
0.623529
0.760784
0.4
0.12549
0.258824
0.670588
0.678431
0.219608
0.219608
0.227451
0.0666667
0.45098
0.52549
0.215686
0.490196
0.0823529
0.172549
0.141176
0.141176
0.164706
0.345098
0.686275
0.658824
0.266667
0.14902
0.172549
0.156863
0.0941176
0.172549
0.556863
0.776471
0.564706
0.215686
0.235294
0.533333
0.717647
0.321569
0.0509804
0.192157
0.160784
0.435294
0.494118
0.254902
